<a href="https://colab.research.google.com/github/Guillermo-rv/RNN/blob/main/rnn_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=10000)

max_length = 200
X_train = pad_sequences(X_train, maxlen=max_length, padding="post")
X_test = pad_sequences(X_test, maxlen=max_length, padding="post")

print(f"✅ Datos cargados → X_train: {X_train.shape}, y_train: {y_train.shape}")


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
✅ Datos cargados → X_train: (25000, 200), y_train: (25000,)


In [ ]:
model = Sequential([
    Input(shape=(200,)),
    Embedding(input_dim=10000, output_dim=128),
    LSTM(128, return_sequences=True),
    Dropout(0.3),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(64, activation="relu"),
    Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 200, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 200, 128)            │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 200, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,465,217 (5.59 MB)

 Trainable params: 1,465,217 (5.59 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
early_stop = EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=10, batch_size=256,
    validation_data=(X_test, y_test),
    class_weight={0: 1.0, 1: 2.0},
    callbacks=[early_stop]
)


Epoch 1/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 11s 54ms/step - accuracy: 0.4999 - loss: 0.9735 - val_accuracy: 0.5000 - val_loss: 0.7474
Epoch 2/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - accuracy: 0.5065 - loss: 0.9438 - val_accuracy: 0.5000 - val_loss: 0.7336
Epoch 3/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - accuracy: 0.5343 - loss: 0.8803 - val_accuracy: 0.6350 - val_loss: 0.6171
Epoch 4/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - accuracy: 0.7256 - loss: 0.6866 - val_accuracy: 0.7686 - val_loss: 0.5360
Epoch 5/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 6s 62ms/step - accuracy: 0.8458 - loss: 0.5200 - val_accuracy: 0.8510 - val_loss: 0.3726
Epoch 6/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 10s 63ms/step - accuracy: 0.8881 - loss: 0.3962 - val_accuracy: 0.8574 - val_loss: 0.3529
Epoch 7/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - accuracy: 0.9022 - loss: 0.3526 - val_accuracy: 0.8528 - val_loss: 0.4055
Epoch 8/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - accuracy: 0.9070 - loss: 0.3220 - val_accuracy: 0.8650 -

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"\n Precisión en conjunto de prueba: {test_acc:.4f}")

plt.plot(history.history["loss"], label="Pérdida Entrenamiento")
plt.plot(history.history["val_loss"], label="Pérdida Validación")
plt.xlabel("Épocas")
plt.ylabel("Pérdida")
plt.legend()
plt.show()

plt.plot(history.history["accuracy"], label="Precisión Entrenamiento")
plt.plot(history.history["val_accuracy"], label="Precisión Validación")
plt.xlabel("Épocas")
plt.ylabel("Precisión")
plt.legend()
plt.show()


231/782 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8655 - loss: 0.3443